# (Interlude) Modularizing Abstraction Code

This notebook is a brief *interlude*: we suspend for the moment our discussion of abstraction to concentrate on coding aspects. The results of this reflections will lay the architecture of the code which will be the bedrock of future work.

## SCMMappings_1_0

So far, we have been implemented code to instantiate abstraction, analyze them, provide insights into them, compute abstraction errors as well as other measures. One by one we have integrated all these functions in a single monolithic class $\mathtt{Abstraction}$ in the *src/SCMMappings.py* file. 

The structure of code looks as follows:

<div style="max-width:600px;">
    <img src="img/UML_SCMMappings_1_0.png" height="100px">
</div>

where we have a three classes:


- **SCMMapping**: the parent class representing a generic SCM mapping. It has two SCMs ($\mathtt{M0},\mathtt{M1}$) as attributes. Its methods include:
    - *outputting functions:* a set of functions  to output and inspect the causal models [list\_\*(), print\_\*(), plot\_\*()] 

- **Abstraction**: a class for $(R,a,\alpha)$ abstraction, inheriting from *SCMMapping*. Its attributes are $\mathtt{R,a,alphas}$. A long list of methods is available, including:
    - *verification functions* used to assess an abstraction is correctly defined [\_are\_()]
    - *property functions* used to access the attributes of an object [is\_\*()]
    - *copying functions* used to duplicare abstractions [copy()]
    - *utility functions* used to perform manipulations on the abstraction [invert\_a(), \_tensorize\_\*(), compute_\*()]
    - *outputting functions:* used to output and inspect the abstraction [list\_\*(), print\_\*(), plot\_\*()] .
    - *evaluating functions:* used to compute quantities related to the abstraction, such as the abstraction error [compute\_\*()]

- **TauOmegaAbstraction**: a preliminary class for $(\tau,\omega)$ abstraction [Rubenstein2017], inheriting from *SCMMapping*. Its attributes are the functions $\mathtt{tau,omega}$ and the intervention sets $\mathtt{I0,I1}$. A few methods have been defined:
    - *verification functions* used to assess an abstraction is correctly defined [_is_order_preserving()]
    - *utility functions* used to perform manipulations on the abstraction [_build_poset()]

## SCMMappings_1_1

Starting from the previous implementation, we refactored the code by externalizing all those functions that are not essential to the various abstraction classes. The classes $\mathtt{SCMMapping}$ and $\mathtt{Abstraction}$ are reduced to the essential functions used to access properties and perform internal manipulations. A few functions are made generic and exported into a *src/utils.py* file. Printing and evaluating functions are externalized in classes that will take care of these functions; these new classes are defined in *src/printing.py* and *src/evaluating.py*, respectively.

The new structure of code looks as follows:

<div style="max-width:900px;">
    <img src="img/UML_SCMMappings_1_1.png" height="100px">
</div>

where we have three main sections:

- **SCMMapping** and the descendants **Abstraction** and **TauOmegaAbstraction** instantiate the abstraction objects, encapsulate the key properties, and include the verification, property, and copying functions.

- **SCMMappingPrinter** and the descendant **AbstractionPrinter** receive an abstraction object, and they provide outputting functions.

- **SCMMappingEvaluator** and the descendant **AbstractionEvaluator** receive an abstraction object, and they provide functions for measuring abstraction quality. Concrete subclasses are **AbstractionErrorEvaluator** (computing the standard abstraction error), **AbstractionInfoLossEvaluator**, and **AbstractionEIEvaluator**.

All the functions are substantially identical, only the overall architecture of the project has changes.

## Testing

We run here a few tests to check the compatibility between the two implementations.

Let us import basic libraries:

In [1]:
import numpy as np

from src.legacy.SCMMappings_1_0 import Abstraction as legacyAbs
from src.SCMMappings_1_1 import Abstraction as newAbs
from src.printing import AbstractionPrinter
from src.evaluating import AbstractionErrorEvaluator
from src.evaluating import AbstractionInfoLossEvaluator
from src.evaluating import AbstractionEffectiveInformationEvaluator

from src.examples import smokingmodels as ex

import sys,io

### Example1

We first testing the identity of printing and evaluation functions in our standard smoking toy example.

In [2]:
M0,M1,R,a,alphas = ex.standardA_M0chainSTC_M1chainSC()

We instantiate a model using the legacy code.

In [3]:
Al = legacyAbs(M0,M1,R,a,alphas)

And a model using the new code.

In [4]:
An = newAbs(M0,M1,R,a,alphas)

We now move on to assess the equality of the printing functions. Notice that for the new model we need to instantiate an *AbstractionPrinter* object.

In [5]:
Ap = AbstractionPrinter(An)

In [6]:
allmethods = dir(AbstractionPrinter)
methods = [m for m in allmethods if (m[0:2]!='__' and m[0:4]!='plot')]

old_stdout = sys.stdout
for m in methods:
    sys.stdout = io.StringIO()
    getattr(Al,m)()
    output1 = sys.stdout.getvalue()
    sys.stdout = io.StringIO()
    getattr(Ap,m)()
    output2 = sys.stdout.getvalue()
    
    assert(output1==output2)
    
sys.stdout = old_stdout

Next we assess the equality in the evaluating functions. As before, for the new model, we instantiate a specific *AbstractionEvaluator* object.

In [7]:
Ae = AbstractionErrorEvaluator(An)

In [8]:
assert(Al.evaluate_abstraction_error() == Ae.evaluate_abstraction_errors())

The two objects behave in the same way.

### Example2

For further confirmation, we run the test on another smoking toy model.

In [9]:
M0,M1,R,a,alphas = ex.standardA_M0chainSC_M1indepSC()

We instantiate old and new objects.

In [10]:
Al = legacyAbs(M0,M1,R,a,alphas)

In [11]:
An = newAbs(M0,M1,R,a,alphas)

We verify the identity for printing functions.

In [12]:
Ap = AbstractionPrinter(An)

In [13]:
allmethods = dir(AbstractionPrinter)
methods = [m for m in allmethods if (m[0:2]!='__' and m[0:4]!='plot')]

for m in methods:
    sys.stdout = io.StringIO()
    getattr(Al,m)()
    output1 = sys.stdout.getvalue()
    sys.stdout = io.StringIO()
    getattr(Ap,m)()
    output2 = sys.stdout.getvalue()
    
    assert(output1==output2)
    
sys.stdout = old_stdout

We verify the identity for evaluating functions.

In [14]:
Ae = AbstractionErrorEvaluator(An)

In [15]:
assert(Al.evaluate_abstraction_error() == Ae.evaluate_abstraction_errors())

The two objects behave in the same way.

## Conclusion

We have reorganized the code to manage our abstraction in a more modular way. The old code will still be available as legacy code in *src/legacy/SCMMappings_1_0.py*. The new code will be available in *src/SCMMappings_1_1.py*.

## Bibliography

[Rischel2020] Rischel, Eigil Fjeldgren. "The Category Theory of Causal Models." (2020).

[Rubenstein2017] Rubenstein, Paul K., et al. "Causal consistency of structural equation models." arXiv preprint arXiv:1707.00819 (2017).